# Simple Data Source Q&A

This is a sample implementation of an analytical Agent for Tableau built with Langgraph

### Dependencies and Environment Variables

First we begin with the dependencies and environment variables needed to run the Agent graph

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# base langchain library imports
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# tableau_langchain imports
from community.langchain_community.tools.tableau.datasource_qa import initialize_datasource_qa

# environment variables available to current process and sub processes
load_dotenv()
# variables for authenticating and interacting with a Tableau site
tableau_domain = os.environ['TABLEAU_DOMAIN']
tableau_site = os.environ['TABLEAU_SITE']
tableau_jwt_client_id = os.environ['TABLEAU_JWT_CLIENT_ID']
tableau_jwt_secret_id = os.environ['TABLEAU_JWT_SECRET_ID']
tableau_jwt_secret = os.environ['TABLEAU_JWT_SECRET']
tableau_api_version = os.environ['TABLEAU_API_VERSION']
tableau_user = os.environ['TABLEAU_USER']
# the target data source for this Tool
datasource_luid = os.environ['DATASOURCE_LUID']
# variables to control LLM models for the Agent and Tools
open_api_key = os.environ["OPENAI_API_KEY"]
agent_llm_model = os.environ['AGENT_MODEL']
tooling_llm_model = os.environ['TOOLING_MODEL']

### Agent Graph

Then we define the Agent graph using Langgraph's prebuilt `create_react_agent` combined with the a tool to query a Tableau data source for conversational analytics

In [2]:
# configure running model for the agent
llm = ChatOpenAI(
    model=agent_llm_model,
    api_key=open_api_key,
    temperature=0,
    verbose=True,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Tableau VizQL Data Service Query Tool
analyze_datasource = initialize_datasource_qa(
    domain=tableau_domain,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model
)

# load the List of Tools to be used by the Agent
tools = [ analyze_datasource ]

# set agent debugging state
if os.getenv('DEBUG') == '1':
    debugging = True
else:
    debugging = False

# define the agent graph
query_agent = create_react_agent(
    model=llm,
    tools=tools,
    debug=debugging
)

### Interacting with the Analytical Agent

After constructing an Analytical Agent with the power to query a Tableau data source, we proceed to ask questions, set tasks and issue commands to the Agent

In [15]:
# question or task sent to the agent
message_string = 'show me average discount, total sales and profits by region sorted by profit'

# Run the agent
messages = query_agent.invoke({"messages": [("human", message_string)]})
agent_message = "\n\nAnalytics Agent:\n\n" + messages['messages'][3].content
display(Markdown(agent_message))



Analytics Agent:

Here are the average discount, total sales, and profits by region, sorted by profit:

| Region  | AVG(Discount) | SUM(Sales)  | SUM(Profit)  |
|---------|---------------|-------------|--------------|
| West    | 0.11          | 739,813.61  | 110,798.82   |
| East    | 0.14          | 691,828.17  | 94,883.26    |
| South   | 0.15          | 391,721.91  | 46,749.43    |
| Central | 0.24          | 503,170.67  | 39,865.31    |

This data shows that the West region has the highest profit, followed by the East, South, and Central regions.